In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from urllib.parse import unquote
from tqdm import tqdm 

dataFolder = '../Data/wikispeedia_paths-and-graph/'
articlesDf = pd.read_csv(dataFolder+ 'articles.tsv')
articlesDf = articlesDf.iloc[10:]
articlesSeries  = pd.Series(articlesDf.index.map(lambda x : x[0])).map(lambda y : unquote(y,'utf-8'))
#articlesSeries.head(10)

linksDf = pd.read_csv(dataFolder + 'links.tsv', header= 10, sep = '\t')
linksDf.index = linksDf.index.map(lambda x : unquote(x, 'utf-8'))
linksDf.columns = ['links']
linksDf['links'] = linksDf['links'].map(lambda x : unquote(x, 'utf-8'))
#linksDf

categoriesDf = pd.read_csv(dataFolder + 'categories.tsv', header=11, sep= '\t')
categoriesDf.index = categoriesDf.index.map(lambda x : unquote(x,'utf-8'))
categoriesDf.columns = ['categories']
#categoriesDf

paths_finishedDf= pd.read_csv(dataFolder + 'paths_finished.tsv', header=15, sep= '\t')
paths_finishedDf.columns = ['hashedIpAddress',   'timestamp',   'durationInSec',  'path',  'rating']
print(paths_finishedDf)

paths_unfinishedDf= pd.read_csv(dataFolder + 'paths_unfinished.tsv', header=16, sep= '\t')
paths_unfinishedDf.columns = ['hashedIpAddress',   'timestamp',   'durationInSec',  'path',  'target', 'type']
#paths_unfinishedDf

        hashedIpAddress   timestamp  durationInSec  \
0      3824310e536af032  1344753412             88   
1      415612e93584d30e  1349298640            138   
2      64dd5cd342e3780c  1265613925             37   
3      015245d773376aab  1366730828            175   
4      5295bca242be81fe  1372890414            110   
...                 ...         ...            ...   
51312  15a13a1d66ef5456  1349231015             66   
51313  2ef7ac844cefda58  1300254138            165   
51314  12863abb7887f890  1385095372            228   
51315  19f8284371753362  1298792567             56   
51316  24b84602179de377  1318122701             30   

                                                    path  rating  
0      14th_century;Europe;Africa;Atlantic_slave_trad...     3.0  
1      14th_century;Niger;Nigeria;British_Empire;Slav...     NaN  
2         14th_century;Renaissance;Ancient_Greece;Greece     NaN  
3      14th_century;Italy;Roman_Catholic_Church;HIV;R...     3.0  
4      14th_cent